In [121]:
import warnings
warnings.filterwarnings("ignore")

# imports best practice pandas
import os

import gzip
import numpy as np
import pandas as pd
import missingno as msno
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
%matplotlib inline
import math
import pickle
import joblib
import dill

#--------------------------------------------------------
# imports best practice sklearn
import sklearn
from sklearn.feature_selection import VarianceThreshold
from sklearn import set_config

from sklearn.tree import DecisionTreeClassifier

# preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder, RobustScaler, MinMaxScaler
from scipy import stats

# transformers
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import FunctionTransformer


from sklearn.cluster import KMeans

# evaluacion
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

# pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
set_config(transform_output = "pandas")

#models
from sklearn.cluster import KMeans

In [122]:
DATA_PATH = "../../data/processed/"
FILE_NAME = "df_capstone_merged.pkl.gz"
PICKLE_PATH = DATA_PATH+FILE_NAME

FILEPATH="../../data/processed/df_capstone_merged.pkl.gz"

with gzip.open(PICKLE_PATH, 'rb') as gz_csv_df:

    df_capstone = pd.read_pickle(gz_csv_df)


In [123]:
def report_df(df, verbose = True):
    '''
    Hace un report simple sobre el DataFrame suministrado.
    '''
    print(df.info(verbose = verbose, show_counts=True))
    total_nulos = df.isnull().sum().sum()
    print()
    print(f"Tenemos un total de {total_nulos} nulos")

In [124]:
report_df(df_capstone)

df_capstone.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   pk_cid              5962924 non-null  int64  
 1   pk_partition        5962924 non-null  object 
 2   country_id          5962924 non-null  object 
 3   region_code         5960660 non-null  float64
 4   gender              5962899 non-null  object 
 5   age                 5962924 non-null  int64  
 6   deceased            5962924 non-null  object 
 7   salary              4450821 non-null  float64
 8   entry_date          5962924 non-null  object 
 9   entry_channel       5829891 non-null  object 
 10  active_customer     5962924 non-null  float64
 11  segment             5828980 non-null  object 
 12  short_term_deposit  5962924 non-null  int64  
 13  loans               5962924 non-null  int64  
 14  mortgage            5962924 non-null  int64  
 15  funds          

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,entry_date,entry_channel,...,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-28,ES,29.0,H,35,N,87218.10,2018-01-12,KHL,...,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-28,ES,13.0,V,23,N,35548.74,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-28,ES,13.0,V,23,N,122179.11,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-28,ES,50.0,H,22,N,119775.54,2015-08-10,KHD,...,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-28,ES,50.0,V,23,N,NaN,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1


In [125]:
df_capstone.tail()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,entry_date,entry_channel,...,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
5962919,1166765,2019-05-28,ES,50.0,V,22,N,43912.17,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962920,1166764,2019-05-28,ES,26.0,V,23,N,23334.99,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962921,1166763,2019-05-28,ES,50.0,H,47,N,NaN,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962922,1166789,2019-05-28,ES,50.0,H,22,N,199592.82,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962923,1550586,2019-05-28,ES,28.0,H,37,N,NaN,2019-05-13,NaN,...,0,0,0,0.0,0.0,0,0,0,0,1


In [126]:
df_capstone.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 27 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   pk_cid              5962924 non-null  int64  
 1   pk_partition        5962924 non-null  object 
 2   country_id          5962924 non-null  object 
 3   region_code         5960660 non-null  float64
 4   gender              5962899 non-null  object 
 5   age                 5962924 non-null  int64  
 6   deceased            5962924 non-null  object 
 7   salary              4450821 non-null  float64
 8   entry_date          5962924 non-null  object 
 9   entry_channel       5829891 non-null  object 
 10  active_customer     5962924 non-null  float64
 11  segment             5828980 non-null  object 
 12  short_term_deposit  5962924 non-null  int64  
 13  loans               5962924 non-null  int64  
 14  mortgage            5962924 non-null  int64  
 15  funds          

In [127]:
def listarNulos(dataset):
    df_stat = dataset.isnull().sum()[(dataset.isnull().sum()>0)].sort_values(ascending=False).to_frame()
    df_stat.rename(columns={0: 'Nulos'}, inplace = True)
    percent = (df_stat["Nulos"]/dataset.shape[0]).mul(100)
    df_stat["porcentaje"] = percent.round(2)
    return df_stat

In [128]:
listarNulos(df_capstone)

,Nulos,porcentaje
salary,1512103,25.36
segment,133944,2.25
entry_channel,133033,2.23
region_code,2264,0.04
payroll,61,0.00
pension_plan,61,0.00
gender,25,0.00


In [129]:
# Función para lista de variables

target = ['']
index = ['']

def obtener_lista_variables(dataset):

  lista_numericas = []
  lista_boolean = []
  lista_categoricas = []

  for i in dataset:
    if ((dataset[i].dtype.kind == 'i') or (dataset[i].dtype.kind == 'f')) and i not in target and i not in index\
    and len(dataset[i].unique()) != 2:
      lista_numericas.append(i)

    elif ((dataset[i].dtype.kind == 'i') or (dataset[i].dtype.kind == 'f') or (dataset[i].dtype.kind == 'b')) and i not in target and i not in index\
    and len(dataset[i].unique()) == 2:
      lista_boolean.append(i)

    elif (dataset[i].dtype.kind == 'O') and i not in target and i not in index:
      lista_categoricas.append(i)

  return lista_numericas, lista_boolean, lista_categoricas

In [130]:
lnum, lbool, lcat = obtener_lista_variables(df_capstone)

In [131]:
print(lnum)

['pk_cid', 'region_code', 'age', 'salary', 'em_account_pp', 'payroll', 'pension_plan']


In [132]:
print(lbool)

['active_customer', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount']


In [133]:
print(lcat)

['pk_partition', 'country_id', 'gender', 'deceased', 'entry_date', 'entry_channel', 'segment']


## Analisis de variables numericas

In [134]:
for i in lnum:
    print(df_capstone[i].value_counts(dropna=False))
    print('')
    print('--------------')
    print('')

1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64

--------------

28.0    1185757
8.0      600362
46.0     358142
30.0     296856
41.0     290447
15.0     254891
29.0     187911
36.0     175850
3.0      171358
11.0     160894
33.0     134562
50.0     132983
6.0      122587
35.0     121982
47.0     116948
18.0     102789
45.0     102328
37.0      92538
10.0      83283
14.0      75405
2.0       72071
21.0      72037
13.0      70092
39.0      69589
12.0      68709
7.0       62436
32.0      54639
27.0      52783
43.0      51782
17.0      50749
9.0       47746
25.0      46887
16.0      36094
24.0      35426
48.0      35256
26.0      34790
4.0       32680
31.0      31241
49.0      30965
23.0      30955
38.0      29001
34.0      27545
19.0      26762
40.0      21658
5.0       21300
22.0      21112
20.0      18833
1.0       12999
44.0      12616
42

In [135]:
# columnas que son realmente booleanas en lugar de numericas
# actualizamos nuestra lista de variables

boolean_instead_of_numeric = ['em_account_pp', 'payroll', 'pension_plan']

for i in boolean_instead_of_numeric:
    
    lnum.remove(i)
    lbool.append(i)

print(lnum)
print(lbool)

['pk_cid', 'region_code', 'age', 'salary']
['active_customer', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'em_account_pp', 'payroll', 'pension_plan']


In [136]:
for i in lnum:
    print(df_capstone[i].value_counts(dropna=False))
    print('')
    print('--------------')
    print('')

1375586    17
1328781    17
1328783    17
1328784    17
1328786    17
           ..
1521045     1
1545224     1
1470941     1
1521049     1
1550586     1
Name: pk_cid, Length: 456373, dtype: int64

--------------

28.0    1185757
8.0      600362
46.0     358142
30.0     296856
41.0     290447
15.0     254891
29.0     187911
36.0     175850
3.0      171358
11.0     160894
33.0     134562
50.0     132983
6.0      122587
35.0     121982
47.0     116948
18.0     102789
45.0     102328
37.0      92538
10.0      83283
14.0      75405
2.0       72071
21.0      72037
13.0      70092
39.0      69589
12.0      68709
7.0       62436
32.0      54639
27.0      52783
43.0      51782
17.0      50749
9.0       47746
25.0      46887
16.0      36094
24.0      35426
48.0      35256
26.0      34790
4.0       32680
31.0      31241
49.0      30965
23.0      30955
38.0      29001
34.0      27545
19.0      26762
40.0      21658
5.0       21300
22.0      21112
20.0      18833
1.0       12999
44.0      12616
42

## Analisis de variables booleanas

In [137]:
for i in lbool:
    print(df_capstone[i].value_counts(dropna=False))
    print('')
    print('--------------')
    print('')

0.0    3557573
1.0    2405351
Name: active_customer, dtype: int64

--------------

0    5947530
1      15394
Name: short_term_deposit, dtype: int64

--------------

0    5962456
1        468
Name: loans, dtype: int64

--------------

0    5962600
1        324
Name: mortgage, dtype: int64

--------------

0    5942825
1      20099
Name: funds, dtype: int64

--------------

0    5940791
1      22133
Name: securities, dtype: int64

--------------

0    5862264
1     100660
Name: long_term_deposit, dtype: int64

--------------

0    5892058
1      70866
Name: credit_card, dtype: int64

--------------

0    5633504
1     329420
Name: payroll_account, dtype: int64

--------------

0    5635963
1     326961
Name: emc_account, dtype: int64

--------------

0    5399956
1     562968
Name: debit_card, dtype: int64

--------------

0    5962890
1         34
Name: em_account_p, dtype: int64

--------------

1    4381602
0    1581322
Name: em_acount, dtype: int64

--------------

0    5962924
Name:

## Analisis de variables categoricas

In [138]:
for i in lcat:
    print(df_capstone[i].value_counts(dropna=False))
    print('')
    print('--------------')
    print('')

2019-05-28    442995
2019-04-28    439627
2019-03-28    436183
2019-02-28    431727
2019-01-28    426875
2018-12-28    422481
2018-11-28    416387
2018-10-28    402300
2018-09-28    375323
2018-08-28    352922
2018-07-28    339339
2018-06-28    252104
2018-05-28    249926
2018-04-28    247463
2018-03-28    245258
2018-02-28    242521
2018-01-28    239493
Name: pk_partition, dtype: int64

--------------

ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
LU         17
ET         17
QA         17
CI         17
SA         17
CM         17
SN         17
MR         17
NO         17
RU         17
CO         17
GA         17
GT         17
DO         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: country_id, d

In [139]:
# columnas que son realmente booleanas en lugar de categoricas
# actualizamos nuestra lista de variables

boolean_instead_of_categoric = ['deceased']

for i in boolean_instead_of_categoric:
    
    lcat.remove(i)
    lbool.append(i)

print(lcat)
print(lbool)

['pk_partition', 'country_id', 'gender', 'entry_date', 'entry_channel', 'segment']
['active_customer', 'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities', 'long_term_deposit', 'credit_card', 'payroll_account', 'emc_account', 'debit_card', 'em_account_p', 'em_acount', 'em_account_pp', 'payroll', 'pension_plan', 'deceased']


In [140]:
df_capstone['deceased'].value_counts(dropna=False)

N    5961849
S       1075
Name: deceased, dtype: int64

In [141]:
def convertir_deceased_a_boolean(df):
    df['deceased'] = df['deceased'].apply(lambda x: 0 if x == 'N' and pd.notnull(x) else 1 if x == 'S' and pd.notnull(x) else x)
    return df

In [142]:
print(df_capstone['deceased'].value_counts(dropna=False))

convertir_deceased_a_boolean(df_capstone)['deceased'].value_counts(dropna=False)

N    5961849
S       1075
Name: deceased, dtype: int64


0    5961849
1       1075
Name: deceased, dtype: int64

In [143]:
deceased_conversion_function = FunctionTransformer(func = convertir_deceased_a_boolean)

In [144]:
deceased_conversion_function.fit_transform(df_capstone)

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,entry_date,entry_channel,...,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount
0,1375586,2018-01-28,ES,29.0,H,35,0,87218.10,2018-01-12,KHL,...,0,0,0,0.0,0.0,0,0,0,0,1
1,1050611,2018-01-28,ES,13.0,V,23,0,35548.74,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
2,1050612,2018-01-28,ES,13.0,V,23,0,122179.11,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
3,1050613,2018-01-28,ES,50.0,H,22,0,119775.54,2015-08-10,KHD,...,0,0,0,0.0,0.0,0,0,0,0,0
4,1050614,2018-01-28,ES,50.0,V,23,0,NaN,2015-08-10,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5962919,1166765,2019-05-28,ES,50.0,V,22,0,43912.17,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962920,1166764,2019-05-28,ES,26.0,V,23,0,23334.99,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962921,1166763,2019-05-28,ES,50.0,H,47,0,NaN,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1
5962922,1166789,2019-05-28,ES,50.0,H,22,0,199592.82,2016-08-14,KHE,...,0,0,0,0.0,0.0,0,0,0,0,1


In [145]:
print(df_capstone['deceased'].value_counts(dropna=False))

deceased_conversion_function.fit_transform(df_capstone)['deceased'].value_counts(dropna=False)

0    5961849
1       1075
Name: deceased, dtype: int64


0    5961849
1       1075
Name: deceased, dtype: int64

In [146]:
# chequeo de que todas las variables fueron revisadas


len(df_capstone.columns) - (len(lnum) + len(lbool) + len(lcat))

0

## Limpieza de nulos

In [147]:
listarNulos(df_capstone)

,Nulos,porcentaje
salary,1512103,25.36
segment,133944,2.25
entry_channel,133033,2.23
region_code,2264,0.04
payroll,61,0.00
pension_plan,61,0.00
gender,25,0.00


In [148]:
# df_capstone['payroll'] = df_capstone.sort_values(["pk_cid", "pk_partition"]).groupby(["pk_cid"])['payroll'].transform(lambda ser: ser.bfill().ffill())
# df_capstone['pension_plan'] = df_capstone.sort_values(["pk_cid", "pk_partition"]).groupby(["pk_cid"])['payroll'].transform(lambda ser: ser.bfill().ffill())

### bfill y ffill para todas las columnas con nulos

In [149]:
def bfill_ffill_for_all_columns_with_NaNs(df,columns):
    df[columns] = df.sort_values(["pk_cid", "pk_partition"]).groupby(["pk_cid"])[columns].transform(lambda ser: ser.bfill().ffill())
    return df

In [150]:
bfill_ffill_for_all_columns_with_NaNs_function = FunctionTransformer(func = bfill_ffill_for_all_columns_with_NaNs,
                                                        kw_args={'columns':['salary','segment',
                                                                            'entry_channel','region_code',
                                                                            'payroll', 'pension_plan','gender']})

In [151]:
df_check = df_capstone.copy()

In [152]:
df_check = bfill_ffill_for_all_columns_with_NaNs_function.fit_transform(df_check)

In [153]:
listarNulos(df_check)

,Nulos,porcentaje
salary,1512103,25.36
entry_channel,9335,0.16
segment,9068,0.15
region_code,2138,0.04
gender,25,0.00


### limpieza de region code

In [154]:
df_capstone[df_capstone['country_id'] == 'ES']['region_code'].isna().sum()

12

In [155]:
df_check[df_check['country_id'] == 'ES']['region_code'].isna().sum()

0

In [156]:
df_capstone[df_capstone['country_id'] != 'ES']['region_code'].isnull().sum()

2252

In [157]:
df_check[df_check['country_id'] != 'ES']['region_code'].isna().sum()

2138

In [158]:
df_impacted_clients = df_capstone.groupby('pk_cid')['country_id'].nunique().sort_values(ascending=False).to_frame().reset_index()

df_impacted_clients[df_impacted_clients['country_id']>1]

,pk_cid,country_id
0,1078010,2
1,1017337,2
2,1050073,2
3,670953,2
4,1381178,2
5,1542679,2
6,1508801,2
7,1282323,2
8,1291853,2
9,1376996,2


In [159]:
df_capstone[df_capstone['pk_cid'] == 1078010][['pk_cid','pk_partition','country_id','region_code']]

,pk_cid,pk_partition,country_id,region_code
58500,1078010,2018-01-28,ES,27.0
414856,1078010,2018-02-28,ES,27.0
712791,1078010,2018-03-28,ES,27.0
910847,1078010,2018-04-28,ES,27.0
1014142,1078010,2018-05-28,ES,27.0
1238877,1078010,2018-06-28,ES,27.0
1506936,1078010,2018-07-28,ES,27.0
2067766,1078010,2018-08-28,ES,27.0
2403485,1078010,2018-09-28,ES,27.0
2683503,1078010,2018-10-28,ES,27.0


In [160]:
df_capstone[df_capstone['region_code'].isna()]['country_id'].unique()

array(['CA', 'CH', 'CL', 'IE', 'AT', 'FR', 'GB', 'DE', 'DO', 'BE', 'AR',
       'VE', 'US', 'MX', 'PL', 'MA', 'GT', 'GA', 'CO', 'BR', 'RU', 'IT',
       'NO', 'SN', 'MR', 'ET', 'CN', 'CM', 'SA', 'CI', 'QA', 'LU', 'ES',
       'SE', 'DJ', 'PT', 'JM', 'RO', 'HU', 'DZ', 'PE'], dtype=object)

In [161]:
df_check[df_check['country_id'] == 'ES']['region_code'].isnull().sum()

0

In [162]:
df_check[df_check['country_id'] != 'ES']['region_code'].isnull().sum()

2138

In [163]:
def imputer_for_region_code(df):
       
    df['region_code'] = df.sort_values(["pk_cid", "pk_partition"]).groupby(["pk_cid"])['region_code'].transform(lambda ser: ser.bfill().ffill())
    
    df['region_code'] = np.where(df['country_id'] != 'ES','Extranjero', df['region_code'])
    
    return df

In [164]:
imputer_for_region_code_function = FunctionTransformer(func = imputer_for_region_code)

In [165]:
df_check2 = df_check.copy()

In [166]:
df_check2 = imputer_for_region_code_function.fit_transform(df_check2)

In [167]:
df_check2[df_check2['country_id'] == 'ES']['region_code'].isnull().sum()

0

In [168]:
df_check2[df_check2['country_id'] != 'ES']['region_code'].isnull().sum()

0

In [169]:
df_check2[df_check2['country_id'] != 'ES']['region_code'].unique()

array(['Extranjero'], dtype=object)

In [170]:
len(df_capstone[df_capstone['country_id'] != 'ES'])

2252

In [171]:
len(df_check2[df_check2['region_code'] == 'Extranjero'])

2252

In [172]:
df_check2[df_check2['country_id'] == 'ES']['region_code'].unique()

array(['29.0', '13.0', '50.0', '45.0', '24.0', '20.0', '10.0', '17.0',
       '49.0', '8.0', '37.0', '9.0', '22.0', '31.0', '5.0', '40.0',
       '27.0', '25.0', '28.0', '3.0', '42.0', '41.0', '39.0', '7.0',
       '47.0', '36.0', '46.0', '44.0', '15.0', '32.0', '23.0', '16.0',
       '48.0', '12.0', '26.0', '2.0', '6.0', '30.0', '11.0', '4.0',
       '19.0', '34.0', '35.0', '14.0', '21.0', '18.0', '33.0', '38.0',
       '52.0', '43.0', '1.0', '51.0'], dtype=object)

In [173]:
len(df_check2[(df_check2['country_id'] == 'ES') & (['region_code']=='Extranjero')])

0

In [174]:
df_check2[df_check2['pk_cid'] == 1078010][['pk_cid','pk_partition','country_id','region_code']]

,pk_cid,pk_partition,country_id,region_code
58500,1078010,2018-01-28,ES,27.0
414856,1078010,2018-02-28,ES,27.0
712791,1078010,2018-03-28,ES,27.0
910847,1078010,2018-04-28,ES,27.0
1014142,1078010,2018-05-28,ES,27.0
1238877,1078010,2018-06-28,ES,27.0
1506936,1078010,2018-07-28,ES,27.0
2067766,1078010,2018-08-28,ES,27.0
2403485,1078010,2018-09-28,ES,27.0
2683503,1078010,2018-10-28,ES,27.0


In [175]:
listarNulos(df_check2)

,Nulos,porcentaje
salary,1512103,25.36
entry_channel,9335,0.16
segment,9068,0.15
gender,25,0.00


### limpieza de entry_date

In [176]:
df_capstone['entry_date'].isnull().sum()

0

In [177]:
df_capstone.groupby('pk_cid')['entry_date'].nunique().sort_values(ascending=False)

pk_cid
1250752    2
383905     2
1083881    2
1534231    2
1542479    2
          ..
1188532    1
1188531    1
1188530    1
1188529    1
1553689    1
Name: entry_date, Length: 456373, dtype: int64

In [178]:
df_capstone[df_capstone['pk_cid'] == 383905][['pk_cid','entry_date']]

,pk_cid,entry_date
2943177,383905,2018-10-24
2954367,383905,2018-11-02
3627812,383905,2018-11-02
4148804,383905,2018-11-02
4479053,383905,2018-11-02
4876516,383905,2018-11-02
5133821,383905,2018-11-02
5523902,383905,2018-11-02


In [179]:
a = '2018-10-24'

In [180]:
a[-5:]

'10-24'

In [181]:
def assign_oldest_entry_date(df):
    
    df['entry_date'] = df['entry_date'].apply(lambda x: str(x)[:5]+'02-28' if(str(x)[-5:]=="02-29") else x)
    df['entry_date'] = pd.to_datetime(df['entry_date'], format = '%Y-%m-%d')
    
    df['entry_date'] = df.sort_values(["pk_cid", "pk_partition"]).groupby(["pk_cid"])['entry_date'].transform(min) 
    
    return df

In [182]:
assign_oldest_entry_date_function = FunctionTransformer(func = assign_oldest_entry_date)

In [183]:
df_check3 = df_check2.copy()

In [184]:
df_check3 = assign_oldest_entry_date_function.fit_transform(df_check3)

In [185]:
df_check3['entry_date'].info()

<class 'pandas.core.series.Series'>
Int64Index: 5962924 entries, 0 to 5962923
Series name: entry_date
Non-Null Count    Dtype         
--------------    -----         
5962924 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 91.0 MB


In [186]:
df_check3[df_check3['pk_cid'] == 383905][['pk_cid','entry_date']]

,pk_cid,entry_date
2943177,383905,2018-10-24
2954367,383905,2018-10-24
3627812,383905,2018-10-24
4148804,383905,2018-10-24
4479053,383905,2018-10-24
4876516,383905,2018-10-24
5133821,383905,2018-10-24
5523902,383905,2018-10-24


In [187]:
listarNulos(df_check3)

,Nulos,porcentaje
salary,1512103,25.36
entry_channel,9335,0.16
segment,9068,0.15
gender,25,0.00


### limpieza de segment y salary

In [188]:
df_check4 = df_check3.copy()

In [189]:
bins = list(range(0, 100, 10))
bins.append(float('inf'))
df_check4['age_cat'] = pd.cut(df_check4["age"],bins=bins)

In [190]:
df_for_imputation = df_check4.groupby(["age_cat"])["segment"].describe().reset_index()

df_for_imputation

,age_cat,count,unique,top,freq
0,"(0.0, 10.0]",19475,1,02 - PARTICULARES,19475
1,"(10.0, 20.0]",443452,3,03 - UNIVERSITARIO,393350
2,"(20.0, 30.0]",3645208,3,03 - UNIVERSITARIO,3415937
3,"(30.0, 40.0]",894172,3,02 - PARTICULARES,711803
4,"(40.0, 50.0]",511336,3,02 - PARTICULARES,481517
5,"(50.0, 60.0]",251713,3,02 - PARTICULARES,228823
6,"(60.0, 70.0]",112767,3,02 - PARTICULARES,98100
7,"(70.0, 80.0]",46013,3,02 - PARTICULARES,37886
8,"(80.0, 90.0]",24775,2,02 - PARTICULARES,19751
9,"(90.0, inf]",4945,2,02 - PARTICULARES,3944


In [191]:
 gbdf =   df_check4.groupby(['age_cat','segment']).agg(
            count_salary = ('salary','count'),
            count_segment = ('segment','count'),
            median_salary = ('salary', 'median'),
            mean_salary = ('salary', 'mean'),
          ).reset_index()
 
gbdf

,age_cat,segment,count_salary,count_segment,median_salary,mean_salary
0,"(0.0, 10.0]",01 - TOP,0,0,NaN,NaN
1,"(0.0, 10.0]",02 - PARTICULARES,15611,19475,122261.760,145975.703161
2,"(0.0, 10.0]",03 - UNIVERSITARIO,0,0,NaN,NaN
3,"(10.0, 20.0]",01 - TOP,229,324,133128.510,188644.140131
4,"(10.0, 20.0]",02 - PARTICULARES,34956,49778,105618.105,138692.722138
5,"(10.0, 20.0]",03 - UNIVERSITARIO,200887,393350,89127.570,112720.846113
6,"(20.0, 30.0]",01 - TOP,908,1240,113914.890,153973.518866
7,"(20.0, 30.0]",02 - PARTICULARES,166468,228031,83327.940,114376.541865
8,"(20.0, 30.0]",03 - UNIVERSITARIO,2628561,3415937,87777.540,115318.770016
9,"(30.0, 40.0]",01 - TOP,13164,17435,109146.750,129371.131338


In [192]:
df_for_imputation = pd.merge(left = df_for_imputation,
                             right = gbdf,
                             how = 'left',
                             left_on = ['age_cat','top'],
                             right_on = ['age_cat','segment'])

df_for_imputation

,age_cat,count,unique,top,freq,segment,count_salary,count_segment,median_salary,mean_salary
0,"(0.0, 10.0]",19475,1,02 - PARTICULARES,19475,02 - PARTICULARES,15611,19475,122261.76,145975.703161
1,"(10.0, 20.0]",443452,3,03 - UNIVERSITARIO,393350,03 - UNIVERSITARIO,200887,393350,89127.57,112720.846113
2,"(20.0, 30.0]",3645208,3,03 - UNIVERSITARIO,3415937,03 - UNIVERSITARIO,2628561,3415937,87777.54,115318.770016
3,"(30.0, 40.0]",894172,3,02 - PARTICULARES,711803,02 - PARTICULARES,537707,711803,84640.77,107844.706819
4,"(40.0, 50.0]",511336,3,02 - PARTICULARES,481517,02 - PARTICULARES,363509,481517,89810.52,115777.665391
5,"(50.0, 60.0]",251713,3,02 - PARTICULARES,228823,02 - PARTICULARES,172237,228823,96539.22,122832.866675
6,"(60.0, 70.0]",112767,3,02 - PARTICULARES,98100,02 - PARTICULARES,74044,98100,101976.87,131909.961669
7,"(70.0, 80.0]",46013,3,02 - PARTICULARES,37886,02 - PARTICULARES,29115,37886,97939.38,138325.818839
8,"(80.0, 90.0]",24775,2,02 - PARTICULARES,19751,02 - PARTICULARES,15844,19751,93684.12,123328.529693
9,"(90.0, inf]",4945,2,02 - PARTICULARES,3944,02 - PARTICULARES,3189,3944,96617.22,165579.692427


In [193]:
df_for_imputation.drop(['segment', 'count_salary', 'count_segment'], axis='columns', inplace=True)

df_for_imputation

,age_cat,count,unique,top,freq,median_salary,mean_salary
0,"(0.0, 10.0]",19475,1,02 - PARTICULARES,19475,122261.76,145975.703161
1,"(10.0, 20.0]",443452,3,03 - UNIVERSITARIO,393350,89127.57,112720.846113
2,"(20.0, 30.0]",3645208,3,03 - UNIVERSITARIO,3415937,87777.54,115318.770016
3,"(30.0, 40.0]",894172,3,02 - PARTICULARES,711803,84640.77,107844.706819
4,"(40.0, 50.0]",511336,3,02 - PARTICULARES,481517,89810.52,115777.665391
5,"(50.0, 60.0]",251713,3,02 - PARTICULARES,228823,96539.22,122832.866675
6,"(60.0, 70.0]",112767,3,02 - PARTICULARES,98100,101976.87,131909.961669
7,"(70.0, 80.0]",46013,3,02 - PARTICULARES,37886,97939.38,138325.818839
8,"(80.0, 90.0]",24775,2,02 - PARTICULARES,19751,93684.12,123328.529693
9,"(90.0, inf]",4945,2,02 - PARTICULARES,3944,96617.22,165579.692427


In [194]:
def imputer_for_missing_salary_and_non_missing_segment(df):
    
  bins = list(range(0, 100, 10))
  bins.append(float('inf'))
  df['age_cat'] = pd.cut(df["age"],bins=bins)
    
  df_for_imputation = df.groupby(["age_cat"])["segment"].describe().reset_index()
    
  gbdf =  df.groupby(['age_cat','segment']).agg(
          count_salary = ('salary','count'),
          count_segment = ('segment','count'),
          median_salary = ('salary', 'median'),
          mean_salary = ('salary', 'mean'),
        ).reset_index()
    
  df_for_imputation = pd.merge(left = df_for_imputation,
                            right = gbdf,
                            how = 'left',
                            left_on = ['age_cat','top'],
                            right_on = ['age_cat','segment'])

    #df_for_imputation.drop(['segment', 'count_salary', 'count_segment'], axis='columns', inplace=True)

    # Caso (1): Registros "salary" = NaN y "segment" != NaN
    # Solución: imputar mediana de "salary" para el "segment" de la edad
    
  filter_aux_1 = (df["salary"].isna()) & (~df["segment"].isna())
    
  for i,idx in enumerate(df[filter_aux_1].index):
      
      age_to_search = df.loc[idx,"age_cat"]
      segment_to_search = df.loc[idx, "segment"]
      salary_to_impute = gbdf.loc[((gbdf["age_cat"]==age_to_search) & (gbdf["segment"]==segment_to_search)), "median_salary"]
      df.loc[idx,"salary_impute_1"] = salary_to_impute.to_list()[0]
  
  df['salary'] = df.apply(lambda row: row['salary'] if not pd.isnull(row['salary']) else row['salary_impute_1'], axis=1)
  
  df.drop(['salary_impute_1'], axis='columns', inplace=True)
        
  return df    

In [195]:
impute_missing_salary_and_non_missing_segment_function = FunctionTransformer(func = imputer_for_missing_salary_and_non_missing_segment)

In [196]:
df_check5 = df_check3.copy()

In [197]:
df_check5 = impute_missing_salary_and_non_missing_segment_function.fit_transform(df_check5)

In [198]:
filter_aux_1 = (df_check5["salary"].isna()) & (~df_check5["segment"].isna())
df_check5[filter_aux_1][["segment", "salary"]]

,segment,salary


In [199]:
listarNulos(df_check5)

,Nulos,porcentaje
entry_channel,9335,0.16
segment,9068,0.15
salary,8820,0.15
gender,25,0.00


In [201]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, r"pickles_check")
FILE_NAME = "df_check5"
PICKLE_PATH = os.path.join(DATA_PATH, FILE_NAME)

df_check5.to_pickle(PICKLE_PATH)

In [202]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, r"pickles_check")
FILE_NAME = "df_check5"
PICKLE_PATH = os.path.join(DATA_PATH, FILE_NAME)

df_check5 = pd.read_pickle(PICKLE_PATH)

In [203]:
listarNulos(df_check5)

,Nulos,porcentaje
entry_channel,9335,0.16
segment,9068,0.15
salary,8820,0.15
gender,25,0.00


In [204]:
def imputer_for_missing_salary_and_missing_segment(df):
    
  bins = list(range(0, 100, 10))
  bins.append(float('inf'))
  df['age_cat'] = pd.cut(df["age"],bins=bins)
    
  df_for_imputation = df.groupby(["age_cat"])["segment"].describe().reset_index()
    
  gbdf =  df.groupby(['age_cat','segment']).agg(
          count_salary = ('salary','count'),
          count_segment = ('segment','count'),
          median_salary = ('salary', 'median'),
          mean_salary = ('salary', 'mean'),
        ).reset_index()
    
  df_for_imputation = pd.merge(left = df_for_imputation,
                            right = gbdf,
                            how = 'left',
                            left_on = ['age_cat','top'],
                            right_on = ['age_cat','segment'])\
                            .set_index('age_cat', drop=True)

    #df_for_imputation.drop(['segment', 'count_salary', 'count_segment'], axis='columns', inplace=True)

    # Caso (2) y (3): Registros "salary" = NaN y "segment" = NaN
    # Solución: imputar moda de "segment" para la edad y la mediana de "salary" para el "segment" de la edad imputado
    
  filter_aux_2_3 = (df["salary"].isna()) & (df["segment"].isna())

  segment_to_impute_2_3_dict = {}
  salary_to_impute_2_3_dict = {}

  for i,idx in enumerate(df[filter_aux_2_3].index):
      age_to_search = df.loc[idx,"age_cat"]
      #segment_to_search = df.loc[idx, "segment"]
      segment_to_impute_2_3 = df_for_imputation.loc[age_to_search, "top"]
      salary_to_impute_2_3 = df_for_imputation.loc[age_to_search, "median_salary"]
      #print(salary_to_imput.to_list()[0])
      #df.loc[idx,"segment_impute_2_3"] = segment_to_impute
      #df.loc[idx,"salary_impute_2_3"] = salary_to_impute
      
      segment_to_impute_2_3_dict[idx] = segment_to_impute_2_3
      salary_to_impute_2_3_dict[idx] = salary_to_impute_2_3
  
  for i in segment_to_impute_2_3_dict.keys():
    df.loc[i,"segment"] = segment_to_impute_2_3_dict[i]
    
  for i in salary_to_impute_2_3_dict.keys():
    df.loc[i,"salary"] = salary_to_impute_2_3_dict[i]
    
  #df['segment'] = df.apply(lambda row: row['segment'] if not pd.isnull(row['segment']) else row['segment_impute_2_3'], axis=1)
  #df['salary'] = df.apply(lambda row: row['salary'] if not pd.isnull(row['salary']) else row['salary_impute_2_3'], axis=1)
  
  #df.drop(['segment_impute_2_3','salary_impute_2_3'], axis='columns', inplace=True)
        
  return df 

In [205]:
impute_missing_salary_and_missing_segment_function = FunctionTransformer(func = imputer_for_missing_salary_and_missing_segment)

In [206]:
df_check6 = df_check5.copy()

In [207]:
df_check6 = impute_missing_salary_and_missing_segment_function.fit_transform(df_check6)

In [208]:
filter_aux_2_3 = (df_check6["salary"].isna()) & (df_check6["segment"].isna())
df_check6[filter_aux_2_3][['segment','salary']]

,segment,salary


In [209]:
listarNulos(df_check6)

,Nulos,porcentaje
entry_channel,9335,0.16
segment,248,0.00
gender,25,0.00


In [210]:
def imputer_for_non_missing_salary_and_missing_segment(df):
    
  bins = list(range(0, 100, 10))
  bins.append(float('inf'))
  df['age_cat'] = pd.cut(df["age"],bins=bins)
    
  df_for_imputation = df.groupby(["age_cat"])["segment"].describe().reset_index()
    
  gbdf =  df.groupby(['age_cat','segment']).agg(
          count_salary = ('salary','count'),
          count_segment = ('segment','count'),
          median_salary = ('salary', 'median'),
          mean_salary = ('salary', 'mean'),
        ).reset_index()
    
  df_for_imputation = pd.merge(left = df_for_imputation,
                            right = gbdf,
                            how = 'left',
                            left_on = ['age_cat','top'],
                            right_on = ['age_cat','segment'])\
                            .set_index('age_cat', drop=True)

    #df_for_imputation.drop(['segment', 'count_salary', 'count_segment'], axis='columns', inplace=True)

    # Caso (4): Registros "segment" = NaN y "salary" != NaN
    # Solución: Calculcar la distancia entre la mediana de "salary" para la moda de "segment" en el rango de edad y 
    # "salary" del cliente y seleccionar el "segment" correspondiente. 
    # distancia = raiz cuadrada el exponente de la diferencia, como forma de evitar valores negativos. 

    # Obs: Podrá haber una pequeña discrepancia en relación a no actualizar la mediana general (afectada por la imputación de los demás valores 
    # anteriores), pero apenas en un numero muy reducido de regsitro = 248
    
  filter_aux_4 = (~df["salary"].isna()) & (df["segment"].isna())

  for i,idx in enumerate(df[filter_aux_4].index):
      age_to_search = df.loc[idx,"age_cat"]
      salary_to_search = df.loc[idx, "salary"]

      # selecionar todos los "segment" para el rango de edad
      df_aux = gbdf[gbdf["age_cat"]==age_to_search]
      
      #calcula distancia entre "salary" y la mediana de "salary" para la moda de "segment" por rango de edad.
      df_aux["dist"] = np.sqrt((df_aux["median_salary"] - salary_to_search)**2)

      #seleciona el "segment" con mediana de "salary" más cercano 
      segment_to_impute = df_aux.loc[(df_aux["dist"] == df_aux["dist"].min()), "segment"]
      
      #Imputa el segmento en la lista
      df.loc[idx,"segment_impute_4"] = segment_to_impute.to_list()[0]
  
  df['segment'] = df.apply(lambda row: row['segment'] if not pd.isnull(row['segment']) else row["segment_impute_4"], axis=1)
  
  df.drop(["segment_impute_4"], axis='columns', inplace=True)
  
  return df
        

In [211]:
impute_non_missing_salary_and_missing_segment_function = FunctionTransformer(func = imputer_for_non_missing_salary_and_missing_segment)

In [212]:
df_check7 = df_check6.copy()

In [213]:
df_check7 = impute_non_missing_salary_and_missing_segment_function.fit_transform(df_check7)

In [214]:
filter_aux_4 = (~df_check7["salary"].isna()) & (df_check7["segment"].isna())

df_check7[filter_aux_4][["salary", "segment","age_cat"]].sort_values(by="age_cat")

,salary,segment,age_cat


In [215]:
listarNulos(df_check7)

,Nulos,porcentaje
entry_channel,9335,0.16
gender,25,0.00


### limpieza de entry channel

In [216]:
gbdf_age_cat_entry_channel = df_check7.groupby(["age_cat"])["entry_channel"].describe().reset_index()

gbdf_age_cat_entry_channel

,age_cat,count,unique,top,freq
0,"(0.0, 10.0]",19475,9,KFC,8994
1,"(10.0, 20.0]",443350,15,KHQ,313973
2,"(20.0, 30.0]",3645067,23,KHE,2790515
3,"(30.0, 40.0]",894078,32,KFC,344167
4,"(40.0, 50.0]",511377,47,KFC,212882
5,"(50.0, 60.0]",251718,46,KFC,88214
6,"(60.0, 70.0]",112768,32,KFC,36485
7,"(70.0, 80.0]",46016,29,KFC,14104
8,"(80.0, 90.0]",24797,23,KFC,7479
9,"(90.0, inf]",4943,14,KFC,1542


In [217]:
def imputer_for_entry_channel(df):
    
  bins = list(range(0, 100, 10))
  bins.append(float('inf'))
  df['age_cat'] = pd.cut(df["age"],bins=bins)
  
  gbdf_age_cat_entry_channel = df.groupby(["age_cat"])["entry_channel"].describe().reset_index()
                                           
  filter_aux = df["entry_channel"].isna()
    
  for i,idx in enumerate(df[filter_aux].index):
      age_to_search = df.loc[idx,"age_cat"]
      entry_channel_to_impute = gbdf_age_cat_entry_channel.loc[(gbdf_age_cat_entry_channel["age_cat"]==age_to_search), "top"]
      #print(ent_channel_to_imput.to_list()[0])
      
      df.loc[idx,"entry_channel"] = entry_channel_to_impute.to_list()[0]
  
  return df

In [218]:
impute_entry_channel_function = FunctionTransformer(func = imputer_for_entry_channel)

In [219]:
df_check8 = df_check7.copy()

In [220]:
df_check8 = impute_entry_channel_function.fit_transform(df_check8)

In [221]:
filter_aux = df_check8["entry_channel"].isna()

df_check8[filter_aux][["age_cat", "entry_channel"]]

,age_cat,entry_channel


In [222]:
listarNulos(df_check8)

,Nulos,porcentaje
gender,25,0.0


### limpieza de gender

In [223]:
def imputer_for_gender(df):
    
  bins = list(range(0, 100, 10))
  bins.append(float('inf'))
  df['age_cat'] = pd.cut(df["age"],bins=bins)
  
  mode_to_impute_gender = df["gender"].mode()[0]
                                           
  df["gender"].fillna(mode_to_impute_gender, inplace=True)
  
  return df

In [224]:
impute_gender_function = FunctionTransformer(func = imputer_for_gender)

In [225]:
df_check9 = df_check8.copy()

In [226]:
df_check9 = impute_gender_function.fit_transform(df_check9)

In [227]:
listarNulos(df_check9)

,Nulos,porcentaje


In [228]:
df_check9.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,entry_date,entry_channel,...,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,age_cat
0,1375586,2018-01-28,ES,29.0,H,35,0,87218.10,2018-01-12,KHL,...,0,0,0.0,0.0,0,0,0,0,1,"(30.0, 40.0]"
1,1050611,2018-01-28,ES,13.0,V,23,0,35548.74,2015-08-10,KHE,...,0,0,0.0,0.0,0,0,0,0,1,"(20.0, 30.0]"
2,1050612,2018-01-28,ES,13.0,V,23,0,122179.11,2015-08-10,KHE,...,0,0,0.0,0.0,0,0,0,0,1,"(20.0, 30.0]"
3,1050613,2018-01-28,ES,50.0,H,22,0,119775.54,2015-08-10,KHD,...,0,0,0.0,0.0,0,0,0,0,0,"(20.0, 30.0]"
4,1050614,2018-01-28,ES,50.0,V,23,0,87777.54,2015-08-10,KHE,...,0,0,0.0,0.0,0,0,0,0,1,"(20.0, 30.0]"


In [229]:
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, r"pickles_check")
FILE_NAME = "df_check9_with_all_data_cleaning"
PICKLE_PATH = os.path.join(DATA_PATH, FILE_NAME)

df_check9.to_pickle(PICKLE_PATH)

### limpieza final de datos

In [230]:
def correct_data_types_and_drop_columns(df):
    
    df['pk_partition'] = pd.to_datetime(df['pk_partition'], format = '%Y-%m-%d')
    df['active_customer'] = df['active_customer'].astype('int64')
    df['payroll'] = df['payroll'].astype('int64')
    df['pension_plan'] = df['pension_plan'].astype('int64')
    
    return df

In [231]:
correct_data_types_and_drop_columns_function = FunctionTransformer(func = correct_data_types_and_drop_columns)

In [232]:
df_check10 = df_check9.copy()

In [233]:
df_check10 = correct_data_types_and_drop_columns_function.fit_transform(df_check10)

In [234]:
df_check10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5962924 entries, 0 to 5962923
Data columns (total 28 columns):
 #   Column              Dtype         
---  ------              -----         
 0   pk_cid              int64         
 1   pk_partition        datetime64[ns]
 2   country_id          object        
 3   region_code         object        
 4   gender              object        
 5   age                 int64         
 6   deceased            int64         
 7   salary              float64       
 8   entry_date          datetime64[ns]
 9   entry_channel       object        
 10  active_customer     int64         
 11  segment             object        
 12  short_term_deposit  int64         
 13  loans               int64         
 14  mortgage            int64         
 15  funds               int64         
 16  securities          int64         
 17  long_term_deposit   int64         
 18  em_account_pp       int64         
 19  credit_card         int64         
 20  pa

## Pipeline

In [235]:
pipe = Pipeline(
    steps = [
        ("deceased_boolean_fix", deceased_conversion_function),                                                             #0
        ("bfill_ffil_for_all_columns_with_NaNs", bfill_ffill_for_all_columns_with_NaNs_function),                           #1
        ("imputer_for_region_code", imputer_for_region_code_function),                                                      #2
        ("assign_oldest_entry_date", assign_oldest_entry_date_function),                                                    #3
        ("imputer_for_missing_salary_and_non_missing_segment", impute_missing_salary_and_non_missing_segment_function),     #4
        ("imputer_for_missing_salary_and_missing_segment", impute_missing_salary_and_missing_segment_function),             #5
        ("imputer_for_non_missing_salary_and_missing_segment", impute_non_missing_salary_and_missing_segment_function),     #6
        ("imputer_for_entry_channel", impute_entry_channel_function),                                                       #7
        ("imputer_for_gender", impute_gender_function),                                                                     #8
        ("correct_data_types_and_drop_columns", correct_data_types_and_drop_columns_function)                               #9              
    ]
)

pipe

Pipeline(steps=[('deceased_boolean_fix',
                 FunctionTransformer(func=<function convertir_deceased_a_boolean at 0x3777de7a0>)),
                ('bfill_ffil_for_all_columns_with_NaNs',
                 FunctionTransformer(func=<function bfill_ffill_for_all_columns_with_NaNs at 0x3777ddb20>,
                                     kw_args={'columns': ['salary', 'segment',
                                                          'entry_channel',
                                                          'region_code',
                                                          'payroll',
                                                          'pension_plan',
                                                          'gender']})...
                 FunctionTransformer(func=<function imputer_for_non_missing_salary_and_missing_segment at 0x1674d14e0>)),
                ('imputer_for_entry_channel',
                 FunctionTransformer(func=<function imputer_for_entry_channel at 0x3777dea20>)),
                ('imputer_for_gender',
                 FunctionTransformer(func=<function imputer_for_gender at 0x3777dd260>)),
                ('correct_data_types_and_drop_columns',
                 FunctionTransformer(func=<function correct_data_types_and_drop_columns at 0x3777dc900>))])

In [236]:
# guardamos las funciones y el pipeline

CWD = os.getcwd()
FUNCTIONS_PATH = os.path.join(CWD, r"functions")

with open(os.path.join(FUNCTIONS_PATH, 'convertir_deceased_a_boolean.joblib'), 'wb') as io:
        dill.dump(convertir_deceased_a_boolean,io)

with open(os.path.join(FUNCTIONS_PATH, 'bfill_ffill_for_all_columns_with_NaNs.joblib'), 'wb') as io:
        dill.dump(bfill_ffill_for_all_columns_with_NaNs,io)

with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_region_code.joblib'), 'wb') as io:
        dill.dump(imputer_for_region_code,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'assign_oldest_entry_date.joblib'), 'wb') as io:
        dill.dump(assign_oldest_entry_date,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_missing_salary_and_non_missing_segment.joblib'), 'wb') as io:
        dill.dump(imputer_for_missing_salary_and_non_missing_segment,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_missing_salary_and_missing_segment.joblib'), 'wb') as io:
        dill.dump(imputer_for_missing_salary_and_missing_segment,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_non_missing_salary_and_missing_segment.joblib'), 'wb') as io:
        dill.dump(imputer_for_non_missing_salary_and_missing_segment,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_entry_channel.joblib'), 'wb') as io:
        dill.dump(imputer_for_entry_channel,io)
        
with open(os.path.join(FUNCTIONS_PATH, 'imputer_for_gender.joblib'), 'wb') as io:
        dill.dump(imputer_for_gender,io)

with open(os.path.join(FUNCTIONS_PATH, 'correct_data_types_and_drop_columns.joblib'), 'wb') as io:
        dill.dump(correct_data_types_and_drop_columns,io)
        
PIPE_PATH = os.path.join(CWD, "pipeline")

pickle.dump(pipe, open(os.path.join(PIPE_PATH, 'pipeline.pkl'), 'wb'))

In [237]:
# guardamos el pipeline

CWD = os.getcwd()

PIPE_PATH = os.path.join(CWD, "pipeline")

pickle.dump(pipe, open(os.path.join(PIPE_PATH, 'pipeline.pkl'), 'wb'))

In [238]:
# cargamos el pipeline

CWD = os.getcwd()

PIPE_PATH = os.path.join(CWD, "pipeline")

pipe = pickle.load(open(os.path.join(PIPE_PATH, 'pipeline.pkl'), 'rb'))